In [1]:
from writers import foyer_xml_writer
from writers.foyer_xml_writer import parmed_to_foyer_xml, mbuild_to_foyer_xml, write_foyer_xml


import ele
import espaloma as esp
import forcefield_utilities as ffutils
import foyer
import gmso
import mbuild as mb
from mbuild.lib.recipes import Polymer
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import numpy as np
from openff.toolkit.topology import Molecule
import torch


import os
import warnings
warnings.filterwarnings("ignore")

if not os.path.exists("espaloma_model.pt"):
    os.system("wget http://data.wangyq.net/espaloma_model.pt")

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


# Example:

### Write out a trimmed-down forcefield file that only has the parameters we're using.

#### This works with our typical workflow:

1. Create molecule or system in mBuild
2. Use an existing focefield along with Foyer to apply the forcefield
3. Write out a useable, trimmed down foyer forcefield file

In [2]:
# mBuild system
alkane = mb.load("CCCCC", smiles=True)
alkane_box = mb.fill_box(alkane, n_compounds=10, box=[2, 2, 2])

# opls forcefield form foyer
#ff = foyer.forcefields.get_forcefield(name="opls")
opls = foyer.Forcefield(name="oplsaa")

# parmed structure
alkane_pmd = opls.apply(alkane_box)

In [3]:
# Save the trimmed xml file:
parmed_to_foyer_xml(
    structure=alkane_pmd,
    ff=opls,
    file_name="alkane_opls.xml",
)

In [4]:
# Test out the xml file we created; we should be able to use it directly with Foyer:
ff_test = foyer.Forcefield(forcefield_files="alkane_opls.xml")
alkane_pmd_test = ff_test.apply(alkane_box)

### Try it out with GAFF

In [5]:
gaff = foyer.forcefields.load_GAFF()
alkane_pmd = gaff.apply(alkane_box)

parmed_to_foyer_xml(
    structure=alkane_pmd,
    ff=gaff,
    file_name="alkane_gaff.xml",
    torsion_type="periodic"
)

gaff_test = foyer.Forcefield(forcefield_files="alkane_gaff.xml")
alkane_gaff_test = gaff_test.apply(alkane_box)

# Example:

### Create a foyer xml file we can use with a abstract system

#### Workflow:
1. Create molecule or system in mBuild
2. Create data structure containing our abstract forcefield parameters
3. Write out a useable Foyer forcefield file

In [6]:
# Example: Creating a bead-spring model of a polymer
mer = mb.Compound()
mer.add(mb.Compound(name="A", pos=[0,0,0], mass=1))
mer.add(mb.Compound(name="A", pos=[0.5,0,0], mass=1))
mer.add(mb.Compound(name="A", pos=[1.0,0,0], mass=1))
mer.freud_generate_bonds(name_a="A", name_b="A", dmax=0.51, dmin=0.049)

chain = Polymer()
chain.add_monomer(mer, indices=[0, 2], replace=False, orientation=[[-1, 0, 0], [1,0,0]], separation=0.5)
chain.build(n=3, add_hydrogens=False)
chain.visualize().show()

box_of_chains = mb.fill_box(compound=chain, n_compounds=10, box=[10, 10, 10])
box_of_chains.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
bond_parameters = {
    ("A", "A"): {"k": 50, "l0": 0.5}
}

angle_parameters = {
    ("A", "A", "A"): {"k": 5, "t0": 2.2}
}

dihedral_parameters = {
    ("A", "A", "A", "A"): {
        "periodicity": [2],
        "k": [10],
        "phase": [3.14]
    }
}

non_bonded_parameters = {
    "A": {"sigma": 1.0, "epsilon": 1.0, "charge": 0}
}

In [8]:
mbuild_to_foyer_xml(
    file_name="testmb.xml",
    compound=box_of_chains,
    bond_params=bond_parameters,
    angle_params=angle_parameters,
    dihedral_params=dihedral_parameters,
    non_bonded_params=non_bonded_parameters,
)

In [9]:
ff_test = foyer.Forcefield(forcefield_files="testmb.xml")
box_of_chains_typed = ff_test.apply(box_of_chains)

In [10]:
snap, hoomd_ff, refs = create_hoomd_forcefield(structure=box_of_chains_typed, auto_scale=False, r_cut=25)

Processing LJ and QQ
No charged groups found, ignoring electrostatics
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions


In [11]:
print(snap.angles.types)
print(snap.dihedrals.types)

['A-A-A']
['A-A-A-A']


In [12]:
print("Pairs:", hoomd_ff[0].params[('A', 'A')])
print("Bonds:", hoomd_ff[3].params["A-A"])
print("Angles:", hoomd_ff[4].params["A-A-A"])
print("Dihedrals:", hoomd_ff[5].params["A-A-A-A"])

Pairs: _HOOMDDict{'epsilon': 0.2390057361376673, 'sigma': 10.0}
Bonds: _HOOMDDict{'k': 0.11950286806883366, 'r0': 5.0}
Angles: _HOOMDDict{'t0': 2.2, 'k': 1.1950286806883366}
Dihedrals: _HOOMDDict{'k': 4.780114722753346, 'd': 1.0, 'n': 2, 'phi0': 3.14}


# Example:

### Fitting espaloma into our MoSDeF-Hoomd Workflow:

In [2]:
# Use espaloma to find the FF parameters for Benzene
smiles = "c1ccccc1"
benzene = mb.load(smiles, smiles=True)
benzene.visualize().show()

molecule = Molecule.from_smiles(smiles)
molecule_graph = esp.Graph(molecule)

espaloma_model = torch.load("espaloma_model.pt")
espaloma_model(molecule_graph.heterograph)
openmm_system = esp.graphs.deploy.openmm_system_from_graph(molecule_graph)

# Store the results for each in something more accessible
pair_forces = openmm_system.getForces()[1]
angle_forces = openmm_system.getForces()[3]
bond_forces = openmm_system.getForces()[2]
torsion_forces = openmm_system.getForces()[0]

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Find the set of particle type parameters:

In [14]:
particle_types = []
particle_type_dict = dict()

for i in range(pair_forces.getNumParticles()):
    pair_parms = pair_forces.getParticleParameters(index=i)
    charge = pair_parms[0]
    sigma = pair_parms[1]
    epsilon = pair_parms[2]
    if (charge, sigma, epsilon) not in particle_types:
        particle_types.append((charge, sigma, epsilon))
    particle_type_dict[i] = particle_types.index((charge, sigma, epsilon))

### `particle_types`  


#### This is the set (i.e. no duplicates) of the particle types found by espaloma

In [15]:
print(len(particle_types))

2


In [19]:
for p in particle_types:
    print(p)
    print()

(Quantity(value=-0.13, unit=elementary charge), Quantity(value=0.3399669508423535, unit=nanometer), Quantity(value=0.359824, unit=kilojoule/mole))

(Quantity(value=0.13, unit=elementary charge), Quantity(value=0.25996424595335105, unit=nanometer), Quantity(value=0.06276, unit=kilojoule/mole))



### `particle_type_dict`  

#### This is a dictionary that points to the parameters for each particle in benzene

#### `key: value`
#### `mbuild particle index: particle types index`

#### We have 12 total particles, but only 2 particle types

In [20]:
len(particle_type_dict)

12

In [21]:
print(particle_type_dict)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1}


In [22]:
for p in particle_type_dict:
    print(particle_type_dict[p])

0
0
0
0
0
0
1
1
1
1
1
1


## Write Foyer XML from Espaloma params

In [3]:
topology = molecule.to_topology()

In [4]:
openmm_topology = topology.to_openmm()

In [5]:
import parmed as pmd
structure = pmd.openmm.load_topology(topology=openmm_topology, system=openmm_system)
structure.bonds.sort(key=lambda x: x.atom1.idx)

In [6]:
atom_types = {}
bond_parameters = {}
angle_parameters = {}
dihedral_parameters = {}
non_bonded_parameters ={}

for bond in structure.bonds:
    bond_type = (bond.atom1.atom_type.name, bond.atom2.atom_type.name)
    if bond_type not in bond_parameters.keys():
        bond_parameters[bond_type] = {"k":bond.type.k , "l0":bond.type.req}
    
for angle in structure.angles:
    angle_type = (angle.atom1.atom_type.name, angle.atom2.atom_type.name, angle.atom3.atom_type.name)
    if angle_type not in angle_parameters.keys():
        angle_parameters[angle_type] = {"k":angle.type.k , "t0":angle.type.theteq}
    
for dihedral in structure.dihedrals:
    dihedral_type = (dihedral.atom1.atom_type.name, dihedral.atom2.atom_type.name,
           dihedral.atom3.atom_type.name, dihedral.atom3.atom_type.name)
    if dihedral_type not in dihedral_parameters.keys():
        dihedral_parameters[dihedral_type] = {"periodicity": [dihedral.type.per], 
                                              "k": [dihedral.type.phi_k], 
                                              "phase": [dihedral.type.phase]}
    
for atom in structure.atoms:
    atom_type = atom.atom_type.name
    if atom_type not in non_bonded_parameters.keys():
        non_bonded_parameters[atom_type] = {"sigma": atom.atom_type.sigma, "epsilon": atom.atom_type.epsilon, "charge": atom.atom_type.charge}
    if atom_type not in atom_types.keys():
        atom_types[atom_type] = {"mass": atom.atom_type.mass}

In [7]:
write_foyer_xml(
    file_name="benzene_esp.xml",
    atom_types=atom_types,
    bond_params=bond_parameters,
    angle_params=angle_parameters,
    dihedral_params=dihedral_parameters,
    non_bonded_params=non_bonded_parameters,
)

In [12]:
for particle in benzene.particles():
    if particle.name.startswith('C'):
        particle.name = "_C1"
    if particle.name.startswith('H'):
        particle.name = "_H1" 

In [15]:
benzene_box = mb.fill_box(benzene, n_compounds=10, box=[2, 2, 2])

In [22]:
ff_benzene_esp = foyer.Forcefield(forcefield_files="benzene_esp.xml")
benzene_box_typed = ff_benzene_esp.apply(benzene_box,  assert_dihedral_params=False)

In [19]:
snap, hoomd_ff, refs = create_hoomd_forcefield(structure=benzene_box_typed, auto_scale=False, r_cut=25)

Processing LJ and QQ
No charged groups found, ignoring electrostatics
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions


In [20]:
print(snap.angles.types)
print(snap.dihedrals.types)

['C1-C1-C1', 'C1-C1-H1']
['C1-C1-C1-C1', 'C1-C1-C1-H1']
